# Data preparation

In [1]:
################################
#Label & Image split and pair  #
#Maintainer: Christopher Chan  #
#Date: 2021-02-05              #
#Version: 0.1.2                #
################################

import os
import pathlib
import argparse
import json
import random
import rasterio as rio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
from zipfile import ZipFile
from glob import glob
from rasterio import features
from rasterio.plot import show
from rasterio.mask import mask
from itertools import product
from PIL import Image

#parser = argparse.ArgumentParser(description="config")
#parser.add_argument("--vector_path", type=str, default="/root/vector",
#                    help="path to project vector file, format: ""/root/Vector""")
#parser.add_argument("--raster_path", type=str, default="/root/raster",
#                    help="path to project raster file, format: ""/root/vector""")
#args = parser.parse_args()

#print("Requested parameters:")

#for arg in vars(args):
#   print("\t", arg, getattr(args, arg))

os.chdir("/home/chris/Dropbox/HOTOSM")

KBY10_path = os.path.abspath("/home/mnt/HOTOSM_data//Kakuma10cm/Kalobeyei")
DZK10_path = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka10cm")
DZKN10_path = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka_N10cm")

KBY15_path = os.path.abspath("/home/mnt/HOTOSM_data//Kakuma15cm/Kalobeyei")
DZK15_path = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka15cm")
DZKN15_path = os.path.abspath("/home/mnt/HOTOSM_data/Dzaleka_N15cm")
#NEN10_path = os.path.abspath("/home/mnt/HOTOSM_data/NE_Nagbo10cm")
code_path = os.path.join(os.getcwd(), "HOTOSM_OAM_codeV2")

ALL_IMG_path = os.path.abspath("/home/chris/Dropbox/HOTOSM/ALL_IMG_15cm")

## Training data Tiles

### Normalisation of separate bands

#### Option A: Calculate using rasterio in memory

In [14]:
with rio.open(os.path.join(DZKN15_path, "Raster","IMask_DZKN15.tif")) as src:
    print(src.name)
    meta = src.meta.copy()
    
    src2 = gdal.Open(src.name)
    R_stats = src2.GetRasterBand(1).GetStatistics(True, True)
    R_min, R_max, R_mean, R_std = R_stats[0], R_stats[1], R_stats[2], R_stats[3]
    
    G_stats = src2.GetRasterBand(2).GetStatistics(True, True)
    G_min, G_max, G_mean, G_std = G_stats[0], G_stats[1], G_stats[2], G_stats[3]
    
    B_stats = src2.GetRasterBand(3).GetStatistics(True, True)
    B_min, B_max, B_mean, B_std = B_stats[0], B_stats[1], B_stats[2], B_stats[3]
   
    # Z-score normalisation
    znorm_R = (src.read(1) - R_mean) / R_std
    znorm_G = (src.read(2) - G_mean) / G_std
    znorm_B = (src.read(3) - B_mean) / B_std
    
    # Linear normalisation of Z-norm
    norm_R = ((znorm_R - znorm_R.min()) * 255) / (znorm_R.max() - znorm_R.min())
    norm_G = ((znorm_G - znorm_G.min()) * 255) / (znorm_G.max() - znorm_G.min())
    norm_B = ((znorm_B - znorm_B.min()) * 255) / (znorm_B.max() - znorm_B.min())
    
    outpath = os.path.join(DZKN15_path, "Raster", "IMask_DZKN15.tif")
    
    with rio.open(outpath, "w", **meta) as out:
        out.write(norm_R, indexes = 1)
        out.write(norm_G, indexes = 2)
        out.write(norm_B, indexes = 3)

/home/mnt/HOTOSM_data/Dzaleka_N15cm/Raster/IMask_DZKN15.tif


#### Option B: If taking too much memory (e.g. KBY)

In [ ]:
gdal_calc.py -A 3857_KBY10cm.tif --format=GTiff --A_band=1 -B 3857_KBY10cm.tif --B_band=2 -C 3857_KBY10cm.tif --C_band=3 --type Float32 --calc="(A-166.6537040821)/29.640548169578" --calc="(B-135.1168216355)/30.53793119212" --calc="(C-104.53429806564)/30.24288788478" --outfile znorm_KBY10cm.tif

gdal_calc.py -A znorm_KBY10cm.tif --format=GTiff --A_band=1 -B znorm_KBY10cm.tif --B_band=2 -C znorm_KBY10cm.tif --C_band=3 --type Float32 --calc="((A--6.1002683639526)*255)/(3.4010300636292--6.1002683639526)" --calc="((B--4.9742159843445)*255)/(4.2805070877075--4.9742159843445)" --calc="((C--4.0053110122681)*255)/(5.3121399879456--4.0053110122681)" --outfile norm_KBY10cm.tif


## GO SPLIT THE RGB AND CREATE STACK.VRT and STACK.TIF WITH RASTERISED_LBL USING SHELL SCRIPTS.

### Cutting into tiles

In [7]:
# Tiling function

def get_tilesA(src, width = 256 , height = 256):
    ncols, nrows = src.meta["width"], src.meta["height"]
    #offsets = product(range(0, ncols, round((width*2)/3)), range(0, nrows, round((height*2)/3)))
    offsets = product(range(0, ncols, width), range(0, nrows, height))
    Big_Tiles = rio.windows.Window(col_off = 0, row_off = 0, width = ncols, height = nrows)
    for col_off, row_off in offsets:
        tiles = rio.windows.Window(col_off = col_off, row_off = row_off, width = width, height = height).intersection(Big_Tiles)
        transform = rio.windows.transform(tiles, src.transform)
        yield tiles, transform

########################################
#                                      #
output_filename = "KBY15_IMG_{}-{}.tif"#
#                                      # 
########################################

with rio.open(os.path.join(KBY15_path, "td_KBY", "stack.tif")) as src:
    tile_width, tile_height = 256, 256
    meta = src.meta.copy()
    
    for tiles, transform in get_tilesA(src):
        #print(tiles)
        meta["transform"] = transform
        meta["width"], meta["height"] = tiles.width, tiles.height
        #outpath = os.path.join(DZKN15_path, "td_DZKN", "IMG", output_filename.format(int(tiles.col_off), int(tiles.row_off)))
        outpath = os.path.join(ALL_IMG_path, output_filename.format(int(tiles.col_off), int(tiles.row_off)))
        with rio.open(outpath, "w", **meta) as out:
            out.write(src.read(window = tiles))
    

In [9]:
# Checkout band 4 to see if label exist
# if sum(sum) of matrix > 0, they exist
# I.e. remove tif with b4 = 0
# Also check if dimension do not confine to 512 by 512 square
# I.e. remove tif if not 512x512 tile

#for root, dirname, filename in os.walk(os.path.join(DZKN15_path, "td_DZKN", "IMG")):
for root, dirname, filename in os.walk(ALL_IMG_path):
    for i in filename:
        with rio.open(root + "/" + i, "r") as img:
            b4 = img.read(4)
            dim = img.shape
            
            if sum(sum(b4)) == 0.0 or dim != (256, 256):
                os.remove(root + "/" + i)

### Replace 2 for 1 in classification for binary segmentation

In [53]:
for root, dirname, filename in os.walk(os.path.join(DZKN10_path, "td_DZKN", "LBL")):
    for i in filename:
        if i.endswith(".png"):
            img = Image.open(root + "/" + i)
            px = img.load()

            width, height = img.size

            for x in range(width):
                for y in range(height):

                    if px[x, y] == 2:
                        px[x, y] = 1

            img.save(os.path.abspath(root + "/" + i))

![False Colour tile sample](/home/mnt/HOTOSM_data/Kakuma/Kalobeyei/td_KBY/IMG/KBY_IMG_34816-12800.png)